## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [23]:
# Load up store_income_data.csv
import pandas as pd
import numpy as np
# Libraries
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

df = pd.read_csv('store_income_data_task.csv')

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [24]:
countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")

# convert to lowercase
df['country'] = df['country'].str.lower()

# Remove trailing white spaces
df['country'] = df['country'].str.strip()
countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 77 unique countries
There are 37 unique countries


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [25]:
# Get the top 10 closest matches to "united kingdom"
matches = fuzzywuzzy.process.extract("uk", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 60):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    # print("All done!")

replace_matches_in_column(df=df, column='country', string_to_match="england")
replace_matches_in_column(df=df, column='country', string_to_match="britain")
replace_matches_in_column(df=df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=df, column='country', string_to_match="america")
replace_matches_in_column(df=df, column='country', string_to_match="united states")
replace_matches_in_column(df=df, column='country', string_to_match="united states of america")
replace_matches_in_column(df=df, column='country', string_to_match="south africa")
replace_matches_in_column(df=df, column='country', string_to_match="u.k.")
replace_matches_in_column(df=df, column='country', string_to_match="uk")
replace_matches_in_column(df=df, column='country', string_to_match="sa")
replace_matches_in_column(df=df, column='country', string_to_match="s.a.")
#replace_matches_in_column(df=df, column='country', string_to_match="/")

# get all the unique values in the 'country' column
# countries = df['country'].unique()

# print(f"There are {len(countries)} unique countries")
# countries

df.replace('britain', 'united kingdom', inplace=True)
df.replace('england', 'united kingdom', inplace=True)
df.replace('u.k.', 'united kingdom', inplace=True)
df.replace('uk', 'united kingdom', inplace=True)
df.replace('united states of america', 'united states', inplace=True)
df.replace('america', 'united states', inplace=True)
df.replace('sa', 'south africa', inplace=True)
df.replace('s.a.', 'south africa', inplace=True)
df.replace('', 'NaN', inplace=True)
df.replace('/', 'NaN', inplace=True)
df.replace('.', 'NaN', inplace=True)

df = df.dropna(subset=['country'])

# Get all the unique values in the 'country' column
countries = df['country'].unique()

countries
print(f"There are {len(countries)} unique countries")
df.head(100)

There are 4 unique countries


,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united kingdom
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united kingdom
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united kingdom
...,...,...,...,...,...,...,...
95,96,Telecom Italia S.P.A.,NaN,Toys,$61199426.97,3-12-2010,south africa
96,97,Bank of America Corporation,zsussans2o@discuz.net,Automotive,$78947711.82,7-12-2006,south africa
97,98,ON Semiconductor Corporation,dmcaneny2p@friendfeed.com,Outdoors,$856142.71,26-3-1963,united kingdom
98,99,International Speedway Corporation,NaN,Automotive,$91817502.46,23-10-2001,united states


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [22]:
# Import modules
from datetime import date

# Convert 'date_measured' to datetime64
df['date_measured'] = pd.to_datetime(df['date_measured'], format='%d-%m-%Y')

# Initialise 'days_ago' column with today's date in datetime64 type
df['days_ago'] = pd.to_datetime(date.today(), format='%d-%m-%Y')

# Subtract the date measured from todays date to get the number of days that have passed
df['days_ago'] = df['days_ago'] - df['date_measured']
df.head()

,id,store_name,store_email,department,income,date_measured,country,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,2006-02-04,united states,6649 days
1,2,Nordson Corporation,NaN,Tools,$41744177.01,2006-01-04,united kingdom,6680 days
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,2003-09-12,united states,7525 days
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,2006-05-08,united kingdom,6556 days
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,1973-01-21,united kingdom,18716 days
